# Sales Analysis

## Import Libraries

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Task - Merge All 12 months worth of sales data into a single dataset

In [53]:
monthly_datasets = []
total_entries = 0

# Read all monthly datasets
for name in [name for name in os.listdir('./Sales_Data')]:
    dataset = pd.read_csv('Sales_Data/' + name)
    monthly_datasets.append(dataset)
    total_entries += dataset.size
    print('Dataset \'{0}\' loaded with {1} entries'.format(name, dataset.size))
    
# Combine them all
sales_data = pd.concat(monthly_datasets)

# Check if everything is loaded
if total_entries == sales_data.size:
    print('\nDatasets merged successfully ({0} rows total).'.format(total_entries))
else:
    print('\nWarning, number of entries don\'t match')

Dataset 'Sales_April_2019.csv' loaded with 110298 entries
Dataset 'Sales_August_2019.csv' loaded with 72066 entries
Dataset 'Sales_December_2019.csv' loaded with 150702 entries
Dataset 'Sales_February_2019.csv' loaded with 72216 entries
Dataset 'Sales_January_2019.csv' loaded with 58338 entries
Dataset 'Sales_July_2019.csv' loaded with 86226 entries
Dataset 'Sales_June_2019.csv' loaded with 81732 entries
Dataset 'Sales_March_2019.csv' loaded with 91356 entries
Dataset 'Sales_May_2019.csv' loaded with 99810 entries
Dataset 'Sales_November_2019.csv' loaded with 105966 entries
Dataset 'Sales_October_2019.csv' loaded with 122274 entries
Dataset 'Sales_September_2019.csv' loaded with 70116 entries

Datasets merged successfully (1121100 rows total).


In [55]:
sales_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
